In [1]:
import serial
import numpy as np
from ParseSerial import ReadSentence_Inclin
import requests
import time
import sys
import h5py
import os
import glob

In [2]:
#Starting with Influx 2.0, authorization tokens are necessary to access databases
#The token can be accessed under the "Data" tab in the Influx UI.

#Stringing together query access
INFLUX_TOKEN='AjsrNgY_k97FMvgfCsgc2tPTx-lOVM-aYaCMjymNVIWpoSCkYh7H4AqIV9pLQHHk07zJa5pxTn4lo-3Ashwu5Q=='
ORG="tim@upenn"
INFLUX_CLOUD_URL='localhost'
BUCKET_NAME='sensors'

QUERY_URI='http://{}:8086/api/v2/write?org={}&bucket={}&precision=ns'.format(INFLUX_CLOUD_URL,ORG,BUCKET_NAME)

headers = {}
headers['Authorization'] = 'Token {}'.format(INFLUX_TOKEN)

#Line protocol
location_tag = 'lab'
measurement_name = 'inclin_data'

In [3]:
# connect to inclinometer
port ='/dev/ttyUSB3'
baudrate=9600
inclin = serial.Serial(port=port, baudrate=baudrate)

In [4]:
header_inclin = "680d0084"
splitline_inclin = [6,12,18]
sentence_length_inclin = 28

In [5]:
#Number of data samples in each save file. (Inclinometer currently set to 5hz)
#N = 1200 * 15
#N = 35100
N = 9000
dataCount = 0

In [6]:
#Creating HDF5 datasets... maybe convert to for loop
timestr = time.strftime("%Y%m%d-%H%M%S")
datestamp = time.strftime("%Y-%m-%d-Inclin")

#First check if there is a folder with current date. If there isn't, create one.
if os.path.isdir("/home/user/tim-daq/data/"+datestamp) == 0:
    os.mkdir("/home/user/tim-daq/data/"+datestamp)

#Then check if the folder is empty. If it's empty, the first file number will start at 1.
#If the folder is not empty, check the file number of the last edited file, and add 1 to it for the new file. 
if len(os.listdir("/home/user/tim-daq/data/"+datestamp)) == 0:
    file_number = 1
    file_number_string = str(1)
else:
    file_list = glob.glob("/home/user/tim-daq/data/"+datestamp+"/*.hdf5")
    latest_file = max(file_list, key=os.path.getctime)
    file_number = int(latest_file[-10:-5]) + 1
    file_number_string = str(file_number)

    
inclinfile = h5py.File("/home/user/tim-daq/data/"+datestamp+"/Inclin_"+timestr+"_"+file_number_string.zfill(5)+".hdf5", "w", libver = 'latest')
x_dset = inclinfile.create_dataset("x", (N,), dtype='float', chunks = True, maxshape = (None,))
y_dset = inclinfile.create_dataset("y", (N,), dtype='float', chunks = True, maxshape = (None,))
temp_dset = inclinfile.create_dataset("temp", (N,), dtype='float', chunks = True, maxshape = (None,))
time_dset = inclinfile.create_dataset("time", (N,), dtype='int64', chunks = True, maxshape = (None,))
index_dset = inclinfile.create_dataset("index", (1,), dtype='int64', chunks = True, maxshape = (None,))
filenames_dset = inclinfile.create_dataset("filenames", (2,), dtype='int64', chunks = True, maxshape = (None,))

filenames_dset[0] = file_number
filenames_dset[1] = file_number + 1

inclinfile.swmr_mode = True

In [7]:
while True:
    
    x_buff, y_buff, temp_buff = ReadSentence_Inclin(inclin, header_inclin, splitline_inclin, sentence_length_inclin) 
    #print(x_buff)
    current_point_time = int(round(time.time() * 1000000000))
    current_line = '{measurement},location={location} inclin_x={x},inclin_y={y},inclin_temp={temp} {timestamp}'.format(measurement=measurement_name,location=location_tag, x=x_buff,y=y_buff,temp=temp_buff, timestamp=current_point_time)       
    
    r = requests.post(QUERY_URI, data=current_line, headers=headers)
    #print(r.status_code)
    
    if dataCount >= N:
        
        inclinfile.close()
        current_timestr = time.strftime("%Y%m%d-%H%M%S")
        datestamp = time.strftime("%Y-%m-%d-Inclin")
        
        if os.path.isdir("/home/user/tim-daq/data/"+datestamp) == 0:
            os.mkdir("/home/user/tim-daq/data/"+datestamp)  
        
        file_list = glob.glob("/home/user/tim-daq/data/"+datestamp+"/*.hdf5")
        latest_file = max(file_list, key=os.path.getctime)
        file_number = int(latest_file[-10:-5]) + 1
        current_file_name = str(file_number)
        
        
        inclinfile = h5py.File("/home/user/tim-daq/data/"+datestamp+"/Inclin_"+current_timestr+"_"+current_file_name.zfill(5)+".hdf5", "w", libver = 'latest')
        
        x_dset = inclinfile.create_dataset("x", (N,), dtype='float', chunks = True, maxshape = (None,))
        y_dset = inclinfile.create_dataset("y", (N,), dtype='float', chunks = True, maxshape = (None,))
        temp_dset = inclinfile.create_dataset("temp", (N,), dtype='float', chunks = True, maxshape = (None,))
        time_dset = inclinfile.create_dataset("time", (N,), dtype='int64', chunks = True, maxshape = (None,))
        index_dset = inclinfile.create_dataset("index", (1,), dtype='int64', chunks = True, maxshape = (None,))
        filenames_dset = inclinfile.create_dataset("filenames", (2,), dtype='int64', chunks = True, maxshape = (None,))

        filenames_dset[0] = file_number
        filenames_dset[1] = file_number + 1
        
        inclinfile.swmr_mode = True
        dataCount = 0
   
    else:
        
        x_dset[dataCount] = x_buff
        y_dset[dataCount] = y_buff
        temp_dset[dataCount] = temp_buff
        time_dset[dataCount] = current_point_time
        index_dset[0] = dataCount
        
        x_dset.flush()
        y_dset.flush()
        temp_dset.flush()
        time_dset.flush()
        index_dset.flush()
        
        
        dataCount += 1
        


KeyboardInterrupt: 

In [ ]:
#for i in np.arange(N):  
    #sn_dset[i], x_dset[i], y_dset[i], z_dset[i], st_dset[i], temp_dset[i] = ReadSentence_Gyro(gyro, header_gyro, splitline_gyro, sentence_length_gyro)
    #print(seq_num, x, y, z, status, temperature)
    #print(sn_dset[i], x_dset[i], y_dset[i], z_dset[i], st_dset[i], temp_dset[i])
    